# JUICE RPWI HF SID22 (PSSR2 rich): L1a QL -- 2024/9/1

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_1-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid22_lib as juice_sid22

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
band_mode = 0                           # 0: sum    1: /Hz
unit_mode = 0                            # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 0                              # 0: linear   1:log  in frequency
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
# cf, p_max0, p_min0, str_unit = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
# print("conversion factor:", cf, "    MAX-min:", p_max0, p_min0, "   str_unit:", str_unit)

# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'                         # Plot dump folder

In [ ]:
# *** Ground data: Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folderdata_name = 'SID21_20231024-0046.cdf'
data_name_list = ['SID22_20231024-0049.cdf',
                 ]
# 202310 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202311_FS/cdf/'    # CDF data folder
data_name_list = ['SID22_Seq06.cdf',
                 ]


In [ ]:
# *** Flight data: Ver.2 ***
# 20240126
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2024/01/26/'        # CDF data folder
data_name_list = ['JUICE_L1a_RPWI-HF-SID22_20240126T091728_V01.cdf',
                 ]
"""
"""

In [ ]:
# *** Group read
"""
data_dir = '/Users/user/0-python/JUICE_data/Data-CDF/2023/05/23/'        # CDF data folder
data_name = '*SID4*.cdf'
cdf_file = data_dir + data_name
data_name_list = glob.glob(cdf_file)
num_list = len(data_name_list)
data_name_list.sort()
for i in range(num_list):
    data_name_list[i] = os.path.split(data_name_list[i])[1]
"""

# get CDF data

In [ ]:
class struct:
    pass

data = struct()
num_list = len(data_name_list)

for i in range(num_list):
    data_name = data_name_list[i]
    cdf_file = data_dir + data_name
    print(i, cdf_file)

    cdf = pycdf.CDF(cdf_file)
    data1 = juice_sid22.hf_sid22_read(cdf)

    if i==0:
        data = data1
        print(data.auto_corr.shape)
    else:
        data = juice_sid22.hf_sid22_add(data, data1)
        print(data.auto_corr.shape)    

data_name = os.path.split(data_name)[1];  print("data name:", data_name)

In [ ]:
"""
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid22.juice_getdata_hf_sid22(cdf)
"""
data = juice_sid22.hf_sid22_shaping(data)

date1 = data.epoch[0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]; date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2;  print(str_date)

n_time = data.auto_corr.shape[0]
n_time0 = n_time-1
print("data size:", data.auto_corr.shape)

In [ ]:
# Mode 
print("==> Ch:", data.ch_selected[0], "  Num-sweep:", n_time, "   Num-AutoCorr: 16    each Length:", data.N_samp_AUX[0] )

# Raw data

In [ ]:
fig = plt.figure(figsize=(14, 11))
ax1 = fig.add_subplot(3, 1, 1);  ax2 = fig.add_subplot(3, 1, 2);  ax3 = fig.add_subplot(3, 1, 3)

ax1.plot(np.ravel(data.auto_corr[:][:]),   linewidth=.5, label='auto_corr')
ax2.plot(np.ravel(data.freq_auto_corr[:]), linewidth=.5, label='frequency')
ax3.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('AutoCorr')
ax2.set_ylabel('Frequency [kHz]')
ax3.set_ylabel('Date-Time')
ax3.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF PSSR2-Rich (SID-22)]  ' + data_name
ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8);  ax2.legend(loc='upper right', fontsize=8)

xlim=[-.5, len(np.ravel(data.auto_corr))     -.5];  print(xlim);  ax1.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.freq_auto_corr))-.5];  print(xlim);  ax2.set_xlim(xlim)
xlim=[-.5, len(np.ravel(data.epoch))         -.5];  print(xlim);  ax3.set_xlim(xlim)
# ylim=[10**p_min0, 10**p_max0];  ax1.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## First data

In [ ]:
n_sweep = 0

fig = plt.figure(figsize=(14, 11))
ax1  = fig.add_subplot(4, 4, 1);  ax2  = fig.add_subplot(4, 4, 2);   ax3  = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5  = fig.add_subplot(4, 4, 5);  ax6  = fig.add_subplot(4, 4, 6);   ax7  = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9  = fig.add_subplot(4, 4, 9);  ax10 = fig.add_subplot(4, 4, 10);  ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14);  ax15 = fig.add_subplot(4, 4, 15); ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time[1:], data.auto_corr[n_sweep][0][1:], linewidth=.5)
ax2.plot(data.time[1:], data.auto_corr[n_sweep][1][1:], linewidth=.5)
ax3.plot(data.time[1:], data.auto_corr[n_sweep][2][1:], linewidth=.5)
ax4.plot(data.time[1:], data.auto_corr[n_sweep][3][1:], linewidth=.5)
ax5.plot(data.time[1:], data.auto_corr[n_sweep][4][1:], linewidth=.5)
ax6.plot(data.time[1:], data.auto_corr[n_sweep][5][1:], linewidth=.5)
ax7.plot(data.time[1:], data.auto_corr[n_sweep][6][1:], linewidth=.5)
ax8.plot(data.time[1:], data.auto_corr[n_sweep][7][1:], linewidth=.5)
ax9.plot(data.time[1:], data.auto_corr[n_sweep][8][1:], linewidth=.5)
ax10.plot(data.time[1:], data.auto_corr[n_sweep][9][1:], linewidth=.5)
ax11.plot(data.time[1:], data.auto_corr[n_sweep][10][1:], linewidth=.5)
ax12.plot(data.time[1:], data.auto_corr[n_sweep][11][1:], linewidth=.5)
ax13.plot(data.time[1:], data.auto_corr[n_sweep][12][1:], linewidth=.5)
ax14.plot(data.time[1:], data.auto_corr[n_sweep][13][1:], linewidth=.5)
ax15.plot(data.time[1:], data.auto_corr[n_sweep][14][1:], linewidth=.5)
ax16.plot(data.time[1:], data.auto_corr[n_sweep][15][1:], linewidth=.5)

date1 = data.epoch[n_sweep];  date1 = date1.strftime('First: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax2.set_title(title_date)
#
ax1.set_xlabel("#1: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][0]))
ax2.set_xlabel("#2: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][1]))
ax3.set_xlabel("#3: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][2]))
ax4.set_xlabel("#4: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][3]))
ax5.set_xlabel("#5: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][4]))
ax6.set_xlabel("#6: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][5]))
ax7.set_xlabel("#7: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][6]))
ax8.set_xlabel("#8: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][7]))
ax9.set_xlabel("#9: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][8]))
ax10.set_xlabel("#10: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][9]))
ax11.set_xlabel("#11: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][10]))
ax12.set_xlabel("#12: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][11]))
ax13.set_xlabel("#13: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][12]))
ax14.set_xlabel("#14: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][13]))
ax15.set_xlabel("#15: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][14]))
ax16.set_xlabel("#16: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][15]))

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_1st.png'
    fig.savefig(png_fname)

## Mid Data

In [ ]:
n_sweep = np.int16(n_time0/2)

fig = plt.figure(figsize=(14, 11))
ax1  = fig.add_subplot(4, 4, 1);  ax2  = fig.add_subplot(4, 4, 2);   ax3  = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5  = fig.add_subplot(4, 4, 5);  ax6  = fig.add_subplot(4, 4, 6);   ax7  = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9  = fig.add_subplot(4, 4, 9);  ax10 = fig.add_subplot(4, 4, 10);  ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14);  ax15 = fig.add_subplot(4, 4, 15); ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time[1:], data.auto_corr[n_sweep][0][1:], linewidth=.5)
ax2.plot(data.time[1:], data.auto_corr[n_sweep][1][1:], linewidth=.5)
ax3.plot(data.time[1:], data.auto_corr[n_sweep][2][1:], linewidth=.5)
ax4.plot(data.time[1:], data.auto_corr[n_sweep][3][1:], linewidth=.5)
ax5.plot(data.time[1:], data.auto_corr[n_sweep][4][1:], linewidth=.5)
ax6.plot(data.time[1:], data.auto_corr[n_sweep][5][1:], linewidth=.5)
ax7.plot(data.time[1:], data.auto_corr[n_sweep][6][1:], linewidth=.5)
ax8.plot(data.time[1:], data.auto_corr[n_sweep][7][1:], linewidth=.5)
ax9.plot(data.time[1:], data.auto_corr[n_sweep][8][1:], linewidth=.5)
ax10.plot(data.time[1:], data.auto_corr[n_sweep][9][1:], linewidth=.5)
ax11.plot(data.time[1:], data.auto_corr[n_sweep][10][1:], linewidth=.5)
ax12.plot(data.time[1:], data.auto_corr[n_sweep][11][1:], linewidth=.5)
ax13.plot(data.time[1:], data.auto_corr[n_sweep][12][1:], linewidth=.5)
ax14.plot(data.time[1:], data.auto_corr[n_sweep][13][1:], linewidth=.5)
ax15.plot(data.time[1:], data.auto_corr[n_sweep][14][1:], linewidth=.5)
ax16.plot(data.time[1:], data.auto_corr[n_sweep][15][1:], linewidth=.5)

date1 = data.epoch[n_sweep];  date1 = date1.strftime('Mid: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax2.set_title(title_date)
#
ax1.set_xlabel("#1: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][0]))
ax2.set_xlabel("#2: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][1]))
ax3.set_xlabel("#3: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][2]))
ax4.set_xlabel("#4: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][3]))
ax5.set_xlabel("#5: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][4]))
ax6.set_xlabel("#6: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][5]))
ax7.set_xlabel("#7: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][6]))
ax8.set_xlabel("#8: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][7]))
ax9.set_xlabel("#9: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][8]))
ax10.set_xlabel("#10: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][9]))
ax11.set_xlabel("#11: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][10]))
ax12.set_xlabel("#12: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][11]))
ax13.set_xlabel("#13: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][12]))
ax14.set_xlabel("#14: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][13]))
ax15.set_xlabel("#15: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][14]))
ax16.set_xlabel("#16: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][15]))

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_mid.png'
    fig.savefig(png_fname)

## Last data

In [ ]:
n_sweep = n_time0

fig = plt.figure(figsize=(14, 11))
ax1  = fig.add_subplot(4, 4, 1);  ax2  = fig.add_subplot(4, 4, 2);   ax3  = fig.add_subplot(4, 4, 3);   ax4 = fig.add_subplot(4, 4, 4)
ax5  = fig.add_subplot(4, 4, 5);  ax6  = fig.add_subplot(4, 4, 6);   ax7  = fig.add_subplot(4, 4, 7);   ax8 = fig.add_subplot(4, 4, 8)
ax9  = fig.add_subplot(4, 4, 9);  ax10 = fig.add_subplot(4, 4, 10);  ax11 = fig.add_subplot(4, 4, 11); ax12 = fig.add_subplot(4, 4, 12)
ax13 = fig.add_subplot(4, 4, 13); ax14 = fig.add_subplot(4, 4, 14);  ax15 = fig.add_subplot(4, 4, 15); ax16 = fig.add_subplot(4, 4, 16)

ax1.plot(data.time[1:], data.auto_corr[n_sweep][0][1:], linewidth=.5)
ax2.plot(data.time[1:], data.auto_corr[n_sweep][1][1:], linewidth=.5)
ax3.plot(data.time[1:], data.auto_corr[n_sweep][2][1:], linewidth=.5)
ax4.plot(data.time[1:], data.auto_corr[n_sweep][3][1:], linewidth=.5)
ax5.plot(data.time[1:], data.auto_corr[n_sweep][4][1:], linewidth=.5)
ax6.plot(data.time[1:], data.auto_corr[n_sweep][5][1:], linewidth=.5)
ax7.plot(data.time[1:], data.auto_corr[n_sweep][6][1:], linewidth=.5)
ax8.plot(data.time[1:], data.auto_corr[n_sweep][7][1:], linewidth=.5)
ax9.plot(data.time[1:], data.auto_corr[n_sweep][8][1:], linewidth=.5)
ax10.plot(data.time[1:], data.auto_corr[n_sweep][9][1:], linewidth=.5)
ax11.plot(data.time[1:], data.auto_corr[n_sweep][10][1:], linewidth=.5)
ax12.plot(data.time[1:], data.auto_corr[n_sweep][11][1:], linewidth=.5)
ax13.plot(data.time[1:], data.auto_corr[n_sweep][12][1:], linewidth=.5)
ax14.plot(data.time[1:], data.auto_corr[n_sweep][13][1:], linewidth=.5)
ax15.plot(data.time[1:], data.auto_corr[n_sweep][14][1:], linewidth=.5)
ax16.plot(data.time[1:], data.auto_corr[n_sweep][15][1:], linewidth=.5)

date1 = data.epoch[n_sweep];  date1 = date1.strftime('Last: %Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1;  ax2.set_title(title_date)
#
ax1.set_xlabel("#1: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][0]))
ax2.set_xlabel("#2: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][1]))
ax3.set_xlabel("#3: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][2]))
ax4.set_xlabel("#4: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][3]))
ax5.set_xlabel("#5: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][4]))
ax6.set_xlabel("#6: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][5]))
ax7.set_xlabel("#7: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][6]))
ax8.set_xlabel("#8: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][7]))
ax9.set_xlabel("#9: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][8]))
ax10.set_xlabel("#10: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][9]))
ax11.set_xlabel("#11: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][10]))
ax12.set_xlabel("#12: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][11]))
ax13.set_xlabel("#13: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][12]))
ax14.set_xlabel("#14: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][13]))
ax15.set_xlabel("#15: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][14]))
ax16.set_xlabel("#16: {0:.0f} [kHz]".format(data.freq_auto_corr[n_sweep][15]))

fig.subplots_adjust(hspace=0)
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_last.png'
    fig.savefig(png_fname)